## 3. Model m2
- With sklearn, several pipelines are built which combine further preprocessing, feature selection, dimensionality reduction, and a modeling algorithm.
- Computations are run with dask, through the joblib backend.

### Setup

In [1]:
from datetime import datetime

from dask.distributed import Client
import joblib

import pandas as pd
import numpy as np

from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [ ]:
class Feature_reader():
    """
    Each feature has a first name, optionally a second name, a statistic and a number.
    This class allows to group feautures according to these aspects or combinations of these aspects.
    Each method produces a list of feature names or a list of lists of feature names.
    """

    def __init__(self, csv):
        self.fts = pd.read_csv(csv, dtype={'n':"string"})
        self.fts = self.fts.fillna('')

    def format(self, select):
        return select.apply(lambda x: '_'.join(x).replace('__', '_'), axis=1).tolist()

    def all(self):
        select = self.fts.copy()
        return self.format(select)

    def first(self):
        select = self.fts.copy()
        select = select.loc[select['n']=='01']
        return self.format(select)

    def min(self):
        select = self.fts.copy()
        select = select.loc[select['stat']=='min']
        return self.format(select)
    
    def max(self):
        select = self.fts.copy()
        select = select.loc[select['stat']=='max']
        return self.format(select)

    def median(self):
        select = self.fts.copy()
        select = select.loc[select['stat']=='median']
        return self.format(select)

    def mean(self):
        select = self.fts.copy()
        select = select.loc[select['stat']=='mean']
        return self.format(select)
    
    def std(self):
        select = self.fts.copy()
        select = select.loc[select['stat']=='std']
        return self.format(select)

    def skew(self):
        select = self.fts.copy()
        select = select.loc[select['stat']=='skew']
        return self.format(select)

    def kurtosis(self):
        select = self.fts.copy()
        select = select.loc[select['stat']=='kurtosis']
        return self.format(select)

    def per_nns(self):
        """
        List of lists per name1, name2, stat, per name1, name2.
        """
        select = self.fts.copy()
        select = [[self.format(grp2) for idx2, grp2 in grp.groupby(by=['stat'], sort=False)] for idx, grp in select.groupby(by=['name1', 'name2'])]
        return select
    
    def per_sn(self):
        """
        List of lists per stat, n. 
        """
        select = self.fts.copy()
        select = [[self.format(grp2) for idx2, grp2 in grp.groupby(by=['n'], sort=False)] for idx, grp in select.groupby(by=['stat'], sort=False)]
        return select

fts = Feature_reader('features.csv')

In [ ]:
y = pd.read_csv(f'featsets/m2/y.csv', header=None)
X = pd.read_csv(f'featsets/m2/X.csv', delimiter=',', header=None)

In [21]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52152,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:52163,Total threads: 2
Dashboard: http://127.0.0.1:52167/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:52158,


### Linear regression

In [ ]:
measures = ['r2', 'neg_mean_squared_error', 'neg_root_mean_squared_error', 'neg_mean_absolute_error']

#### Linear regression with all fts

In [24]:
with joblib.parallel_backend('dask'):
    pipeline = Pipeline([
        ('reg', LinearRegression()),
    ])
    # print(pipeline.get_params().keys())

    parameters = {
        'reg__fit_intercept': [False, True]
    }

    grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, scoring=measures, refit=False)
    grid_search.fit(X, y)

    results = pd.DataFrame(grid_search.cv_results_)

frmt(results)

/Users/juliet/Documents/UGENT/2021-2022/big_data/practicals/venv/lib/python3.8/site-packages/sklearn/model_selection/_search.py:978: RuntimeWarning: overflow encountered in square
  (array - array_means[:, np.newaxis]) ** 2, axis=1, weights=weights


,r2,mse,rmse,mae
params,,,,
{'reg__fit_intercept': False},-0.255,3.994728e+303,6.168976e+151,4.010090e+151
{'reg__fit_intercept': True},-0.254,3.993815e+303,6.168506e+151,4.009533e+151


#### Linear regression with pca reduction

In [25]:
with joblib.parallel_backend('dask'):
    pipeline = Pipeline([
        ('pca', PCA()),
        ('reg', LinearRegression()),
    ])
    # print(pipeline.get_params().keys())

    parameters = {
        'pca__n_components': [5, 10, 20],
    }

    grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, scoring=measures, refit=False)
    grid_search.fit(X, y)

    results = pd.DataFrame(grid_search.cv_results_)

frmt(results)

/Users/juliet/Documents/UGENT/2021-2022/big_data/practicals/venv/lib/python3.8/site-packages/sklearn/model_selection/_search.py:978: RuntimeWarning: overflow encountered in square
  (array - array_means[:, np.newaxis]) ** 2, axis=1, weights=weights


,r2,mse,rmse,mae
params,,,,
{'pca__n_components': 5},0.015,3.143468e+303,5.606488e+151,4.169702e+151
{'pca__n_components': 10},0.019,3.129532e+303,5.594056e+151,4.153914e+151
{'pca__n_components': 20},0.030,3.095833e+303,5.563876e+151,4.139734e+151


#### Linear regression with pca and polynomial fts

In [26]:
with joblib.parallel_backend('dask'):
    pipeline = Pipeline([
        ('pca', PCA()),
        ('poly', PolynomialFeatures()),
        ('reg', LinearRegression()),
    ])

    parameters = {
        'pca__n_components': [5, 10, 20],
        'poly__degree': [2]
    }

    grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, scoring=measures, refit=False)
    grid_search.fit(X, y)

    results = pd.DataFrame(grid_search.cv_results_)

frmt(results)

/Users/juliet/Documents/UGENT/2021-2022/big_data/practicals/venv/lib/python3.8/site-packages/sklearn/metrics/_regression.py:805: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
/Users/juliet/Documents/UGENT/2021-2022/big_data/practicals/venv/lib/python3.8/site-packages/sklearn/metrics/_regression.py:442: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
/Users/juliet/Documents/UGENT/2021-2022/big_data/practicals/venv/lib/python3.8/site-packages/sklearn/metrics/_regression.py:442: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
/Users/juliet/Documents/UGENT/2021-2022/big_data/practicals/venv/lib/python3.8/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, wh

,r2,mse,rmse,mae
params,,,,
"{'pca__n_components': 5, 'poly__degree': 2}",0.022,3.120193e+303,5.585697e+151,4.152045e+151
"{'pca__n_components': 10, 'poly__degree': 2}",0.039,3.064597e+303,5.535731e+151,4.109533e+151
"{'pca__n_components': 20, 'poly__degree': 2}",-inf,inf,inf,4.195975e+151


### Ridge regression
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge


### Lasso

- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLarsCV.html#sklearn.linear_model.LassoLarsCV

### Tree regression

- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.ExtraTreeRegressor.html#sklearn.tree.ExtraTreeRegressor
- https://george-jen.gitbook.io/data-science-and-apache-spark/decision-tree-regression

### Close client

In [27]:
client.close()